In [2]:
from bitstring import BitArray
import numpy as np

def hexToDec(hex_num):
    hex_num = hex_num[::-1]
    hex_num = hex_num.hex()
    hex_num = '0x' + hex_num
    hex_num = BitArray(hex=hex_num)
    return hex_num.uint

def find_names(file, dir_cluster):
    while True:
        curr_f = file.read(32)
        if curr_f[0:1] == b'\x00':
            break
        else:
            print(curr_f[0:11])
        
file = open('boot622.IMA', 'rb')
mft = file.read(512)

#считывание mft
sector = hexToDec(mft[11:13])
claster = hexToDec(mft[13:14])
copies = hexToDec(mft[16:17])
objects = hexToDec(mft[17:19])
num_of_sectors = hexToDec(mft[19:21])
FAT_size = hexToDec(mft[22:24])
print("Загрузочная запись:", "\nсектор:", sector, "\nкластер:", claster, "\nкопии FAT:", copies, "\nобъекты:", objects, "\nразмер директории:", num_of_sectors, "\nразмер FAT в секторах:", FAT_size)

#вычисление объема файлов
FAT = file.read(sector * FAT_size)
FATarray = np.frombuffer(FAT, dtype=np.uint8)
nonzero = np.nonzero(FATarray)[0]
file_volume = nonzero.size * claster * sector
print("Объем, занимаемый файлами:", file_volume)


#вывод списка файлов
FAT = file.read(sector * FAT_size)
find_names(file, claster + copies * FAT_size)

#поиск фалй scandiskini
file.seek((claster + copies * FAT_size) * sector)
file_name = b'SCANDISKINI'
address = ''
while True:
        curr_f = file.read(32)
        if curr_f[0:11] == file_name:
            volume = hexToDec(curr_f[28:32])
            address = curr_f[20:22][::-1] + curr_f[26:28][::-1]
            address = address[::-1]
            address = hexToDec(address)
            print(curr_f[0:11], address, volume)
            break
#вывод содержимого файла
file.seek((claster + copies * FAT_size * claster + int(objects * 32 / sector * claster) + address - 2) * sector)
text = file.read(volume);
print("Содержимое файла:\n",text)

file.close()

FileNotFoundError: [Errno 2] No such file or directory: 'boot622.IMA'